# Import

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace

# Hyperparameter

In [2]:
config = {
    "learning_rate": 2e-5,
    "epoch": 10,
    "batch_size": 64
}

CFG = SimpleNamespace(**config)

# Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBT해커톤/Data/train_df_1008.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBT해커톤/Data/test_df_1008.csv")

In [5]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54314 entries, 0 to 54313
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54314 non-null  object
 1   분류      54314 non-null  object
 2   제목      54314 non-null  object
 3   키워드     54314 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
dtypes: object(3)
memory usage: 548.7+ KB


# Load Model

In [6]:
# # 토크나이저와 모델 변경
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import BertTokenizer, RobertaForSequenceClassification

In [7]:
CFG = SimpleNamespace(**config)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=len(train_df['분류'].unique())).to(device)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
if torch.cuda.is_available():
    print(True)
else:
    print(False)

True


# Custom Dataset

In [10]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=200): # 클래스 초기화 (__init__ 메서드)
        self.texts = texts                                      # texts: 텍스트 데이터의 리스트입니다.
        self.labels = labels                                    # labels: 각 텍스트에 대한 레이블(정답)의 리스트입니다. 레이블이 없는 경우를 위해 기본값을 None으로 설정할 수 있습니다.
        self.tokenizer = tokenizer                              # tokenizer: 텍스트를 토큰화하는 데 사용할 토크나이저 객체입니다. 주로 BERT와 같은 모델의 토크나이저를 사용합니다.
        self.max_len = max_len                                  # max_len: 토큰화된 텍스트의 최대 길이입니다. 기본값은 128로 설정되어 있습니다.

    def __len__(self):                                          # 데이터셋 길이 (__len__ 메서드)
        return len(self.texts)                                  # 데이터셋의 크기를 반환합니다. 즉, 텍스트의 개수를 반환합니다.

    def __getitem__(self, item):                                # 데이터 항목 가져오기 (__getitem__ 메서드)
        text = str(self.texts[item])                            # item: 데이터셋에서 가져올 특정 인덱스입니다.
        label = self.labels[item] if self.labels is not None else -1 # label: 해당 인덱스의 레이블을 가져옵니다. 레이블이 없으면 -1로 설정합니다.
        encoding = self.tokenizer.encode_plus(                  # encoding: 토크나이저의 encode_plus 메서드를 사용하여 텍스트를 인코딩합니다. 이 과정에서 다음과 같은 옵션을 설정합니다:
            text,                                              #
            add_special_tokens=True,                            # add_special_tokens=True: 모델에 필요한 특별한 토큰을 추가합니다.
            max_length=self.max_len,                            # max_length=self.max_len: 최대 길이를 설정합니다.
            return_token_type_ids=False,                        #
            padding='max_length',                               # padding='max_length': 최대 길이에 맞춰 패딩을 추가합니다.
            truncation=True,                                    # truncation=True: 최대 길이를 초과하는 경우 잘라냅니다.
            return_attention_mask=True,                         # return_attention_mask=True: 어텐션 마스크도 반환합니다.
            return_tensors='pt',                                # return_tensors='pt': PyTorch 텐서 형식으로 반환합니다.
        )
        return {                                                # 반환하는 딕셔너리에는 원본 텍스트, 인코딩된 입력 ID, 어텐션 마스크, 레이블이 포함
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# Data Preprocessing

In [11]:
# 데이터 준비
# train_df['키워드'] = train_df['제목'] + ' ' + train_df['키워드']
# test_df['키워드'] = test_df['제목'] + ' ' + test_df['키워드']

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)   # train_df에서 키워드와 label을 리스트로 변환하여 TextDataset 클래스의 인스턴스를 생성
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)         # val_df에서 키워드와 label을 사용하여 검증 데이터셋을 생성
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)                        # test_df에서 키워드를 리스트로 변환하여 TextDataset을 생성합니다. 이 데이터셋은 레이블이 없으므로 None을 전달

# 데이터 로더 생성, DataLoader는 데이터를 미니 배치 단위로 나누어서 제공해주는 역할
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [12]:
train_dataset[30]

{'text': '아파트 판서 신원 리얼돌 체험방 리얼돌 체험방 전국적 성행 논란 제작진 취재결과 리얼돌 체험방 간판 비밀리 인천시 청라동 리얼돌 체험방 제작진 정확 체험방 리얼돌 체험방 계좌번호 송금 문자 도착 연락 이용호수 안내 간판 오피스텔 간판 입금 주소 출입문 비밀 약속 장소 전화 리얼돌 체험방 도착 부동산 입구 표시 일반 오피스텔 내부 리얼돌 체험방 예약 체험방 진입 나이 신분증 체험방 중고생 이용 환경 중고생 리얼돌 체험방 세대 한복판 아파트 단지 아이들 학원가 블록 스톱워치 거리 미터 오피스텔 주민 정말 대박 사건 상상 사람들 상상 주민 인근 아파트 반대 카페 원래 얘기 간판 카페 반대 운동 남자 호기심 걱정 엄마들 공인중개사 주민들 반대 매춘 반대 상가 오피스텔 실제 국민 청원 등장 주민들 반대 영업 중단 오피스텔 비밀 영업 인근 주민 관할 경찰 인천 경찰 업소 본인 영업 평온 주택가 아파트 단지 리얼돌 체험관 주민들 반대 영업 강행 신원 나이 위험천만 영업 행태 주민들 당혹감 표정',
 'input_ids': tensor([    0,  4155, 22907, 12728,  7913,  2324,  4303,  2239,  7913,  2324,
          4303,  2239,  3952,  2125, 21797,  4496,  9713,  6081,  2489,  2145,
          7913,  2324,  4303,  2239,  8366, 23217,  7722, 16675,  2328,  7913,
          2324,  4303,  2239,  9713,  4480,  4303,  2239,  7913,  2324,  4303,
          2239,  6964,  2517,  2016, 12373,  5703,  5082,  5721,  3774,  2016,
          2113,  5611,  8366,  8787,  8366, 12793,  7862, 18408,  5361,  4680,
 

위의 텐서는 자연어 처리에서 토큰화된 텍스트를 나타내며, 각 숫자는 특정 토큰에 대한 ID를 의미한다. 출력된 텐서는 모델 학습이나 예측에 사용한다.

In [13]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
# # 학습
# model.train()
# for epoch in range(CFG.epoch):
#     for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#     # Validation
#     model.eval()
#     val_predictions = []
#     val_true_labels = []
#     with torch.no_grad():
#         for batch in tqdm(val_loader, desc='Validating'):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids, attention_mask=attention_mask)
#             _, preds = torch.max(outputs.logits, dim=1)
#             val_predictions.extend(preds.cpu().tolist())
#             val_true_labels.extend(labels.cpu().tolist())

#     # 검증 결과 출력
#     val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
#     print(f"Validation F1 Score: {val_f1:.2f}")

In [15]:
import numpy as np

# 초기 설정
best_f1 = -np.inf  # 가장 좋은 F1 점수 초기화
patience = 3  # 개선이 없을 경우 멈추기 전 허용할 에포크 수
counter = 0  # 개선이 없었던 에포크 수 카운터

model.train()
for epoch in range(CFG.epoch):
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())

    # 검증 결과 출력
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation F1 Score: {val_f1:.2f}")

    # F1 점수가 개선되었는지 확인
    if val_f1 > best_f1:
        best_f1 = val_f1
        counter = 0  # 개선이 있었으므로 카운터 초기화
    else:
        counter += 1  # 개선이 없으므로 카운터 증가

    # 개선이 없었던 에포크 수가 patience를 초과하면 학습 중단
    if counter >= patience:
        print("Early stopping triggered.")
        break

Validating: 100%|██████████| 170/170 [01:32<00:00,  1.85it/s]


Validation F1 Score: 0.46


Validating: 100%|██████████| 170/170 [01:31<00:00,  1.85it/s]


Validation F1 Score: 0.58


Validating: 100%|██████████| 170/170 [01:31<00:00,  1.85it/s]


...

Validation F1 Score: 0.65


Validating: 100%|██████████| 170/170 [01:31<00:00,  1.85it/s]


Validation F1 Score: 0.63


Validating: 100%|██████████| 170/170 [01:31<00:00,  1.85it/s]

Validation F1 Score: 0.63


# Inference

In [16]:
# 테스트 세트 추론
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 366/366 [03:18<00:00,  1.85it/s]


# Submission

In [17]:
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBT해커톤/Data/sample_submission.csv")
submission["분류"] = decoded_predictions

submission.to_csv("/content/drive/MyDrive/Colab Notebooks/GBT해커톤/submission/1009_roberta.csv", encoding='UTF-8-sig', index=False)

In [19]:
submission['분류'].value_counts()

,count
분류,
지역,11656
경제:부동산,1492
사회:사건_사고,1137
경제:반도체,991
사회:사회일반,515
사회:교육_시험,463
경제:취업_창업,387
사회:의료_건강,385
정치:국회_정당,365
